In [ ]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(cowplot))
suppressMessages(library(dplyr))
suppressMessages(library(tidyr))
suppressMessages(library(ComplexHeatmap))
suppressMessages(library(RColorBrewer))

In [ ]:
obj.atac <- readRDS("../data/VisiumHeart/snATAC.annotated.Rds")
obj.atac

In [ ]:
## add gene activity matrix
gene.activities <- readRDS("../data/VisiumHeart/GeneScoreMatrix.Rds")

gene.activities <- gene.activities[, colnames(obj.atac)]

obj.atac[['GeneActivity']] <- CreateAssayObject(counts = gene.activities)

DefaultAssay(obj.atac) <- "GeneActivity"

obj.atac <- obj.atac %>% 
        NormalizeData() %>%
        FindVariableFeatures() %>%
        ScaleData() %>%
        RunPCA(verbose = FALSE)

In [ ]:
Idents(obj.atac) <- "cell_type"

In [ ]:
all.markers <- FindAllMarkers(obj.atac, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
all.markers$cluster <- factor(all.markers$cluster, levels = c("CM", "Fib", "Endo", "Lymphoid", "Myeloid", "Pericyte", "vSMCs", "Neuronal"))

In [ ]:
all.markers <- all.markers[order(all.markers$cluster), ]

In [ ]:
df <- all.markers %>%
    group_by(cluster) %>%
    slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
## get average gene activity score
avg <- AverageExpression(
  obj.atac,
  assays = "GeneActivity",
  features = df$gene,
  return.seurat = FALSE,
  group.by = "cell_type",
  slot = "data",
  verbose = TRUE,
)


In [ ]:
df_plot <- avg$GeneActivity

head(df_plot)

In [ ]:
df_plot_scale <- t(apply(df_plot, 1, scale))
colnames(df_plot_scale) <- colnames(df_plot)
rownames(df_plot_scale) <- rownames(df_plot)

In [ ]:
head(df_plot_scale)

df_plot_scale <- df_plot_scale[, c("CM", "Fib", "Endo", "Lymphoid", "Myeloid", "Pericyte", "vSMCs", "Neuronal")]

In [ ]:
df_plot_scale <- df_plot_scale[rev(rownames(df_plot_scale)), ]

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 14)

p <- Heatmap(as.matrix(df_plot_scale),
             name = "Gene Activity",
             cluster_columns = FALSE,
             cluster_rows = FALSE,
             show_row_names = TRUE,
             rect_gp = gpar(col = "black", lwd = 0.5),
             col = ArchR::paletteContinuous()
            )

p

In [ ]:
saveRDS(df_plot_scale, "../data/VisiumHeart/marker_heatmap.Rds")

In [ ]:
## Session information
sessionInfo()